In [ ]:
#!pip install geemap
!pip install ndvi2gif

In [2]:
import geemap
import ee
from ndvi2gif import NdviSeasonality

In [4]:
 ee.Authenticate()
ee.Initialize(project='ee-digdgeografo')

In [13]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

We utilize the map drawing tools to create a roi

In [32]:
roi = Map.draw_last_feature

In [16]:
# Here we select ndwi max for 12 periods seasons in Andalucia
myclass = NdviSeasonality(roi=roi, sat='S2', key='mean', periods=12,start_year=2017, end_year=2022, index='ndwi')
s2 = myclass.get_year_composite()

vizParams = {'bands': ['january', 'march', 'september'], 'min': 0, 'max':0.7}

Map.addLayer(s2, vizParams, 'Median')

There we go again...


In [31]:
Map = geemap.Map(height='1500px')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [33]:
# Doñana NDVI
# Here we select ndwi max for 12 periods seasons
# Show graphics and pixel values
# Also change to 24 periods

myclass = NdviSeasonality(roi=roi, sat='S2', key='max', periods=12,start_year=2022, end_year=2024, index='ndvi')
s2 = myclass.get_year_composite()

vizParams = {'bands': ['january', 'april', 'october'], 'min': 0, 'max':0.8}

Map.addLayer(s2, vizParams, 'Max')

There we go again...


Of course, it's possible to use loops to load several data at once

In [34]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [35]:
roi = Map.draw_last_feature

In [36]:
# Statistics loop

stats = ['max', 'median', 'mean', 'perc_90', 'perc_95']
vizParams = {'bands': ['march', 'april', 'june'], 'min': 0, 'max':0.8}

for i in stats:

    myclass = NdviSeasonality(roi=roi, sat='S2', key=i, periods=12,start_year=2020, end_year=2022, index='ndvi')
    s2 = myclass.get_year_composite()
    Map.addLayer(s2, vizParams, str(i))

There we go again...
There we go again...
There we go again...
There we go again...
There we go again...


In [37]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [38]:
# Years loop
# Change roi to Doñana deimsID

years = [i for i in range(2018, 2023)]

vizParams = {'bands': ['march', 'april', 'june'], 'min': 0, 'max':0.8}

for y in years:

    name = 'maxNDVI_' + str(y)
    myclass = NdviSeasonality(roi=roi, sat='S2', key='perc_90', periods=12,start_year=y, end_year=y+1, index='ndvi')
    s2 = myclass.get_year_composite()
    Map.addLayer(s2, vizParams, name)

There we go again...
There we go again...
There we go again...
There we go again...
There we go again...


We could do something bigger.
Africa Example with MODIS

In [43]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [44]:
import ee
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq("wld_rgn", "Africa"))

In [45]:
# Africa NDVI Mean
# Also show Nile delta with S2, take a sight on what you can see over the desert

Africa = NdviSeasonality(roi=countries, sat='MODIS', key='mean', periods=4,start_year=2020, end_year=2022, index='ndvi')
MODIS = Africa.get_year_composite()

vizParams = {'bands': ['summer', 'winter', 'spring'], 'min': 0.1, 'max':0.8}
Map.addLayer(MODIS, vizParams, 'S2_P95_24')

There we go again...


We said deimsPY allows to select your eLTER site qith only using your deimsID, so let's check it out

In [46]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [47]:
# Braila Island Flood

myclass = NdviSeasonality(roi='deimsid:https://deims.org/d4854af8-9d9f-42a2-af96-f1ed9cb25712', sat='S2', key='perc_95', periods=12,start_year=2017, end_year=2022, index='ndwi')
s2 = myclass.get_year_composite()
vizParams = {'bands': ['october', 'june', 'january'], 'min': 0.1, 'max':0.8}
Map.addLayer(s2, vizParams, 'Braila Island P95')

There we go again...
Con DEIMS hemos topado amigo Sancho...


Let's try to get the maximum flood in al januarys

In [48]:
# Max january flood

januaryFloodmax = s2.select('january').max()
Map.addLayer(januaryFloodmax, {'min': 0, 'max':0.8}, 'January Max Flood P90')

In [49]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
roi = Map.draw_last_feature

In [50]:
# Doñana NDVI Stats
myclass = NdviSeasonality(roi='deimsid:https://deims.org/bcbc866c-3f4f-47a8-bbbc-0a93df6de7b2', sat='Landsat',
                          key='max', periods=12,start_year=2017, end_year=2023, index='ndvi')
Landsat = myclass.get_year_composite()
vizParams = {'bands': ['january', 'april', 'september'], 'min': 0.1, 'max':0.8}

Map.addLayer(Landsat, vizParams, 'Max')

There we go again...
Con DEIMS hemos topado amigo Sancho...


We can also have zonal statistic

In [51]:
# Points and polygons zonal stats

import os

# This is a Google Earth Engine Feature Collection
alcornoques = ee.FeatureCollection('users/digdgeografo/pajarera')

# This is a local point shapefile
samples = '/home/diego/samples.shp'
geom = geemap.shp_to_ee(samples)

# Define the ouptut shape
out_shp = os.path.join(os.getcwd(), 'mystatstets_oaks.shp')

# Compute the statistics
geemap.zonal_statistics(Landsat, in_zone_vector=alcornoques, out_file_path=out_shp, statistics_type='MEAN', scale=30)

No module named 'pycrs'
Computing statistics ...
Generating URL ...
Please wait ...
An error occurred while downloading. 
 Retrying ...
Generating URL ...
Please wait ...
Data downloaded to /content/mystatstets_oaks.shp


It is also possible to export the collections. There's a problem qith size limits from GEE but small areas are ok

In [52]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [53]:
# Take a smaller roi

roi = Map.draw_last_feature

In [56]:
# Here we select ndwi max for 12 periods seasons in Andalucia

myclass = NdviSeasonality(roi=roi, sat='S2', key='max', periods=4,start_year=2020, end_year=2023, index='ndvi')
Landsat = myclass.get_year_composite()
vizParams = {'bands': ['spring', 'summer', 'autumn'], 'min': 0.1, 'max':0.8}

Map.addLayer(Landsat, vizParams, 'Max')

There we go again...


In [57]:
# Export it and load the data in QGIS

myclass.get_export()

3
Exporting /content/ndvi_2020.tif 

Generating URL ...
Please wait ...
Data downloaded to /content/ndvi_2020.tif
Exporting /content/ndvi_2021.tif 

Generating URL ...
Please wait ...
Data downloaded to /content/ndvi_2021.tif
Exporting /content/ndvi_2022.tif 

Generating URL ...
Please wait ...
Data downloaded to /content/ndvi_2022.tif
All the images in the collection have been exported


In [58]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [59]:
roi = Map.draw_last_feature

In [60]:
# Last but not least and exaample of Sierra Nevada NDSI

myclass = NdviSeasonality(roi=roi, sat='S2', key='median',
                          periods=12,start_year=2020, end_year=2022, index='ndsi')
s2 = myclass.get_year_composite()

vizParams = {'bands': ['january', 'february', 'march'], 'min': 0, 'max':0.8}
Map.addLayer(s2, vizParams, 'S2_MAX_Sierra Nevada')

There we go again...
